<a href="https://colab.research.google.com/github/fwitschel/WIMA/blob/main/notebooks/Expert_RAG_IRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Execute this code only if in colab
if 'COLAB_GPU' in os.environ:
  print("Executing in Colab!")
  # Cloning GitHub repository
  !git clone https://github.com/fwitschel/WIMA.git
  %cd WIMA


Executing in Colab!
Cloning into 'QDMKM'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 49 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 45.29 KiB | 2.52 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/QDMKM



First, we will install required dependencies.

In [1]:
!pip install langchain langchain-community pypdf sentence_transformers faiss-cpu langchain-anthropic groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IRF_Data.csv', sep=";")
df.head()

,person,dc.title,dc.subtitle
0,"Grunder, Hans-Ulrich","Der Bildungsforschung die Daten, der Allgemein...",NaN
1,"Kirnbauer, Martin",NaN,NaN
2,"Hell, Benedikt",Entwicklung eines Interviewleitfadens für die ...,NaN
3,"Herrmann, Anne",Entwicklung eines Interviewleitfadens für die ...,NaN
4,"Labudde, Peter",Didaktische Leitfragen für die Lehrerinnen- un...,Beiträge zur Lehrerbildung (BZL)


### Parsing the documents
First, we will load the csv document and parse it using Pandas.

In [5]:
from langchain_core.documents import Document
docs = []
for index, row in df.iterrows():
    person = row['person']
    title = row['dc.title']
    document = Document(
        page_content= "Author: " + str(person) + ", Title: " + str(title),
        metadata={"source": "IRF"}
    )
    docs.append(document)

print(docs[0])

page_content='Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik' metadata={'source': 'IRF'}


In [6]:
print(docs[0].page_content[0:250])

Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik


Once we have the text from the document, we have to split it into smaller chunks. We can use LangChain's available splitters, like CharacterTextSplitter in this case:

In [7]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunks = splitter.split_documents(docs)
print(chunks[0])

page_content='Author: Grunder, Hans-Ulrich, Title: Der Bildungsforschung die Daten, der Allgemeinen Pädagogik die skeptischen RückFragen? Bildungsforschung und Allgemeine Pädagogik' metadata={'source': 'IRF'}


We will be using BGE-small, an opensource embeddings model. We will download it from HuggingFace Hub and run it on all chunks to calculate their vector representations.

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

chunk_texts = list(map(lambda d: d.page_content, chunks))
embeddings = bge_embeddings.embed_documents(chunk_texts)
print(embeddings[0])

/tmp/ipython-input-1782701561.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  bge_embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Once we have the vector representations for all chunks, we can create an in-memory vector database and store all vectors in it. For this example, we will be using a FAISS database.

In [ ]:
from langchain_community.vectorstores import FAISS

text_embedding_pairs = zip(chunk_texts, embeddings)
db = FAISS.from_embeddings(text_embedding_pairs, bge_embeddings)

The database is now set up. Now, we will be taking queries from the user on this information. In the case of expert search, we expect the user to ask for an area of expertise. Then, we retrieve the top k most similar chunks to that query.

In [ ]:
topk = 42
query = "data science"

contexts = db.similarity_search(query, k=topk)

for i in range(topk):
  print(contexts[i].page_content)


Author: Göldi, Susan, Title: Big Data Scientist und App-Entwicklerin statt Laternenträger und Kutscher
Author: Herzog, Daniel, Title: Big Data Scientist und App-Entwicklerin statt Laternenträger und Kutscher
Author: Mareis, Claudia, Title: Datenspuren. Big Data im Kontext von Kultur und Gesellschaft
Author: Kilchmann, Daniel, Title: Monitoring statistical data preparation
Author: Boes, Stefan (external), Title: The Leaky Pipeline in the Swiss University System: Identifying Gender Barriers in Postgraduate Education and Networks Using Longitudinal Data.
Author: Mareis, Claudia, Title: Das Ende der Anschaulichkeit. Künstlerisch-gestalterische Strategien im Umgang mit Big Data
Author: Witschel, Hans Friedrich, Title: Data-Mining im KMU - Das Beste aus Daten.
Author: Vavouras, Athanasios (external), Title: Data Warehousing: Concepts and Mechanisms.
Author: Leemann, Regula Julia, Title: The Leaky Pipeline in the Swiss University System: Identifying Gender Barriers in Postgraduate Education a

After retrieving the relevant context, we build a prompt using this information and the user's original query. We will use a Llama model via the Huggingface API:



> This example uses Huggingface API to call the model. In order for it to work, remember to set the Secret Variable "HUGGINGFACE_API_KEY" to your own Huggingface API Key, or change the model to any of your choice.



In [ ]:
from groq import Groq
def llm(groq_client, prompt):
  chat_completion = groq_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
  )

  return chat_completion.choices[0].message.content

In [ ]:
groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY')
)

In [ ]:
context = '\n\n'.join(list(map(lambda c: c.page_content, contexts)))
prompt = f"""You are an assistant that helps users to find people with a certain expertise, based on their publications.
You need to analyse the publications and check who has most relevant publications in the given field! The following context contains several relevant publications;
for each publication, the author is provided first: {context}. Please only return internal experts!
The field that we are interested in is {query}"""
response = llm(groq_client, prompt)
print(response)

To determine who is an expert in the field of data science, I will analyze the provided publications and identify the authors with the most relevant publications in this field.

After analyzing the publications, I found that the following internal authors have published works related to data science:

1. Mareis, Claudia (3 publications: "Datenspuren. Big Data im Kontext von Kultur und Gesellschaft", "Das Ende der Anschaulichkeit. Künstlerisch-gestalterische Strategien im Umgang mit Big Data", "Big Data in the Context of Culture and Society")
2. Himmelsbach, Sabine (2 publications: "Das Ende der Anschaulichkeit. Künstlerisch-gestalterische Strategien im Umgang mit Big Data", "Big Data in the Context of Culture and Society")
3. Göldi, Susan (1 publication: "Big Data Scientist und App-Entwicklerin statt Laternenträger und Kutscher")
4. Herzog, Daniel (1 publication: "Big Data Scientist und App-Entwickelerin statt Laternenträger und Kutscher")
5. Kilchmann, Daniel (1 publication: "Monitori